In [1]:
import os
import pickle
import random

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
os.chdir("../data")

In [3]:
base_dir = 'item_correlations'

In [4]:
anime = pd.read_csv("AnimeList.csv")
anime = anime[["anime_id", "title"]]

In [5]:
df = pd.read_csv("UserAnimeList.csv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/kundan/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 2155, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 847, in pandas._libs.parsers.TextReader.read
  File "pandas/_libs/parsers.pyx", line 862, in pandas._libs.parsers.TextReader._read_low_memory
  File "pandas/_libs/parsers.pyx", line 941, in pandas._libs.parsers.TextReader._read_rows
  File "pandas/_libs/parsers.pyx", line 1073, in pandas._libs.parsers.TextReader._convert_column_data
  File "pandas/_libs/parsers.pyx", line 1119, in pandas._libs.parsers.TextReader._convert_tokens
  File "pandas/_libs/parsers.pyx", line 1162, in pandas._libs.parsers.TextReader._convert_with_dtype
  File "/Users/kundan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/dtypes/common.py", line 530, in is_categorical_dtype
    def is_categorical_dtype(arr_or_dtype) -> bool:
KeyboardInterrupt

During handling of the above exception, another ex

TypeError: object of type 'NoneType' has no len()

In [ ]:
len(df["username"].unique()), len(df["anime_id"].unique())

In [ ]:
filtered_df = df[["username", "anime_id", "my_score"]].loc[lambda x: x["my_score"] != 0]

In [ ]:
average_rating = filtered_df["my_score"].mean()
user_bias = (
    pd.DataFrame(filtered_df.groupby("username")["my_score"].mean()).rename(
        {"my_score": "user_bias"}, axis=1
    )
    - average_rating
)
anime_bias = (
    pd.DataFrame(filtered_df.groupby("anime_id")["my_score"].mean()).rename(
        {"my_score": "anime_bias"}, axis=1
    )
    - average_rating
)

In [ ]:
filtered_df = filtered_df.merge(anime_bias, on=["anime_id"]).merge(
    user_bias, on=["username"]
)

In [ ]:
filtered_df["normalized_score"] = (
    filtered_df["my_score"]
    - filtered_df["anime_bias"]
    - filtered_df["user_bias"]
    - average_rating
)

In [ ]:
filtered_df = filtered_df.set_index("anime_id")

In [ ]:
filtered_df = filtered_df.dropna()

In [ ]:
filtered_df

In [ ]:
def get_corrs(recommendees):
    item_subset = (
        filtered_df.loc[recommendees]
        .reset_index()
        .merge(filtered_df.reset_index(), on="username")
    )

    adj_cos_corr_numerator = item_subset.groupby(["anime_id_x", "anime_id_y"]).apply(
        lambda x: np.dot(x["normalized_score_x"], x["normalized_score_y"])
    )
    adj_cos_corr_denom = filtered_df.groupby("anime_id").apply(
        lambda x: np.sqrt(np.dot(x["normalized_score"], x["normalized_score"]))
    )
    x_length = adj_cos_corr_denom[recommendees]
    x_length.index.rename("anime_id_x", inplace=True)
    y_length = adj_cos_corr_denom
    y_length.index.rename("anime_id_y", inplace=True)
    adj_cos_corr = adj_cos_corr_numerator / x_length / y_length
    adj_cos_corr = pd.DataFrame(adj_cos_corr, columns=["corr"]).dropna()

    raw_corrs = adj_cos_corr
    corr_sizes = item_subset.groupby(["anime_id_x", "anime_id_y"]).size()
    corrs = raw_corrs.merge(
        pd.DataFrame(corr_sizes, columns=["size"]), on=["anime_id_x", "anime_id_y"]
    )
    corrs["similarity"] = corrs["corr"].abs()
    
    # We assume variance is the same as the variance for pearson correlation.    
    corrs = corrs.loc[lambda x: x["size"] > 2]
    corrs["corr_var"] = (1 - corrs["corr"] * corrs["corr"]) ** 2 / (corrs["size"] - 2)    
    corrs = corrs.sort_values(by="similarity").dropna()
    return corrs[["corr", "corr_var", "size"]]

In [ ]:
anime_ids = sorted(list(set(filtered_df.index)))

In [ ]:
# Store correlations on disk. We need to do this in chunks
# for memory reasons
chunks = np.array_split(anime_ids, len(anime_ids) / 10)
for i, chunk in tqdm(enumerate(chunks)):
    corr = get_corrs(chunk)
    corr.to_pickle(os.path.join(base_dir, f"{i}.pkl"))

In [ ]:
corr_dfs = []
for item in tqdm(os.listdir(base_dir)):
    path = os.path.join(base_dir, item)
    corr_dfs.append(pickle.load(open(path, "rb")))

In [ ]:
corr_df = pd.concat(corr_dfs)

In [ ]:
corr_df.to_pickle(os.path.join(base_dir, "correlations.pkl"))